# Scientific Tools in Python

## 1. Equation Solving using `scipy.optimize.fsolve`

In [3]:
from scipy.optimize import fsolve

#### Example 1: Solve
$$
x^2 =2
$$

In [2]:
f01=lambda x: x**2-2

def f02(x):
    return x**2-2

x=fsolve(f01, 5)
print(x)

x=fsolve(f02, 1)
print(x)

[1.41421356]
[1.41421356]


#### Define `myfzero`

In [3]:
def myfzero(func, x0):
    return fsolve(func, x0)[0]
    pass 

In [4]:
f01=lambda x: x**2-2
def f02(x):
    return x**2-2

x=myfzero(f01, 1)
print(x)

x=myfzero(f02, 1)
print(x)

1.4142135623730947
1.4142135623730947


In [5]:
# How to obtain the negative root of 
#             x**2-2=0
# Method 1
x=fsolve(f02, -1)[0]
print(x)
# Method 2
from scipy.optimize import bisect
x=bisect(f02, -10, -0.5)
print(x)

-1.4142135623730947
-1.4142135623723675


#### Functions that return a function/functions (1)

In [6]:
def f1(x):
    return 2*x
def f2(x):
    return x**2
def flist(x, y):
    fun_list=[f1, f2]
    return fun_list[(x+y)%2]
f=flist(1,4)
print(f(3))

fun=lambda a, b: lambda x: a*x+b
f=fun(2,3)
print(f(4))

9
11


#### Functions that return a function/functions (2)

In [7]:
def flist2():
    return [lambda x: i*x for i in [3, 5]]        
f3,f5=flist2()
print(f(5), f5(5))

13 25


In [8]:
#What is the output?
i=3
f=lambda x: i*x
i=5
print(f(5))
i=7
print(f(5))

25
35


#### Example 2: Given `fxab(x,a,b)`, solve `fxab(x,2,3)=0`.

In [7]:
import numpy as np
def fxab(x, a, b):
    return a*x**3+b
x=fsolve(fxab, 1, args=(2, 3))[0]
x

-1.1447142425533323

In [10]:
a = 2
b = 3
x = fsolve(lambda x:fxab(x, a, b), 1)
print(x)

[-1.14471424]


In [10]:
(-3/2)**(1/3)

(0.572357121276666+0.991351614125047j)

In [11]:
-(3/2)**(1/3)

-1.1447142425533319

In [7]:
def mynthroot(x, n):
    if n%2 ==0:return (x)**(1/n)
    else:
        if x>=0:return (x)**(1/n)
        else:return -(-x)**(1/n)
mynthroot(-3/2, 3)

-1.1447142425533319

In [13]:
a=2
b=3
x=fsolve(lambda x: fxab(x,a,b), 1)[0]
x

-1.1447142425533323

#### Find Implied Volatility of an European Call Option

$$ BS(S, K, r, q, \sigma, T)= S \cdot e^{-qT} \cdot \Phi (d_1) - K e^{-rT} \cdot \Phi (d_2)$$

where 

$$ \displaystyle d_1 = \frac{\ln \left( \frac{S}{K} \right) + \left( r-q+\frac{\sigma ^2 }{2}\right) T}{\sigma \sqrt{T}}, \quad d_2 = d_1 - \sigma \sqrt{T} $$

In [10]:
from scipy.stats import norm
from math import log, sqrt, exp
def BS_EuroCall(S,K,r,q,sigma,T):
   d1=(log(S/K)+(r-q+sigma**2/2)*T)/(sigma*sqrt(T))
   d2=d1-sigma*sqrt(T)
   c=S*exp(-q*T)*norm.cdf(d1)-K*exp(-r*T)*norm.cdf(d2)
   return c

S=490
K=470
r=0.033
q=0
T=0.08
sigma=0.2
c=BS_EuroCall(S,K,r,q,sigma,T)
print(c)

24.5940926130354


#### Example 3: Given $S=490$, $K=470$, $r=0.033$, $q=0$, $T=0.08$ and $c=24.5941$, use `scipy.optimize.fsolve` / `myfzero` and `BS_EuroCall` to solve the Black-Scholes equation for the implied volatilty ($\sigma$).

In [165]:
S=490
K=470
r=0.033
q=0
T=0.08
c = 24.59410
IV = fsolve(lambda x : BS_EuroCall(S,K,r,q,x,T)-c,1)[0]
IV

0.20000018572007525

In [12]:
S=490
K=470
r=0.033
q=0
T=0.08
BS1=lambda x: BS_EuroCall(S, K, r, q, x, T)
BS1(0.2)

24.5940926130354

In [169]:
from functools import partial
S=490
K=470
r=0.033
q=0
T=0.08
c=24.5941
sigma = 0.2
BS2=partial(BS_EuroCall, S, K, T=T)
BS2(0.033, 0, 0.2)

24.5940926130354

In [97]:
sigma_imp=fsolve(lambda x: BS2(x)-c, 0.05)
print(sigma_imp)

[0.20000019]


In [171]:
def mypartial(fun, *args, **kwargs):
    def f(x):
        return fun(*args, x, **kwargs)
    return f
BS3=mypartial(BS_EuroCall, S, K, r, q, T=T)
c=BS3(0.2)
sigma_imp=fsolve(lambda x: BS3(x)-c, 0.5)
print(sigma_imp)

[0.2]


#### Example 4: Compute the implied volatility for each row of `data`.

In [21]:
import pandas as pd
from scipy.optimize import fsolve
# Load data from dataset01.csv
data=pd.read_csv('dataset01.csv')
data

,S,K,r,q,sigma,T,c
0,1403,1350,0.0534,0.0118,0.260,0.102778,80.828
1,1403,1375,0.0534,0.0118,0.267,0.102778,66.084
2,1403,1400,0.0534,0.0118,0.231,0.102778,45.894
3,1403,1425,0.0534,0.0118,0.213,0.102778,30.955
4,1403,1450,0.0534,0.0118,0.198,0.102778,19.224


In [174]:
#Compute Implied Volatility for each row, and put the result in a new column
data[['S', 'K', 'r', 'q', 'T', 'c']].apply(
      lambda x: fsolve(lambda s: BS_EuroCall(*(x[0:4]),s,x[4])-x[5], 0.5), axis=1)

0    [0.26000099756907497]
1    [0.26699897872478867]
2    [0.23099920350758713]
3    [0.21299749074903945]
4    [0.19799965471169964]
dtype: object

In [25]:
data['ImpVol'] = data[['S', 'K', 'r', 'q', 'T', 'c']].apply(
    lambda x: fsolve(lambda s: 
                     BS_EuroCall(*(x[0:4]),s,x[4])-x[5], 0.5)[0], axis=1)
data

,S,K,r,q,sigma,T,c,ImpVol
0,1403,1350,0.0534,0.0118,0.260,0.102778,80.828,0.260001
1,1403,1375,0.0534,0.0118,0.267,0.102778,66.084,0.266999
2,1403,1400,0.0534,0.0118,0.231,0.102778,45.894,0.230999
3,1403,1425,0.0534,0.0118,0.213,0.102778,30.955,0.212997
4,1403,1450,0.0534,0.0118,0.198,0.102778,19.224,0.198000


In [83]:

data.to_csv('output.csv')

In [24]:
#Compute Implied Volatility for each row, and put the result in a new column
#f=lambda s: fsolve(lambda x: BS_EuroCall(s[0],s[1],s[2],s[3],x,s[4])-s[5], 0.5)
#data['ImpVol']=data.loc[:,data.columns!='sigma'].apply(f, axis=1, result_type='expand')
#t

#### Example 5: Solve the following system of nonlinear equations.
\begin{align*}
  x_0 \cos (x_1) &= 4 \\
  x_0 x_1 -x_1 &= 5
\end{align*}

In [25]:
from math import cos
def f03(x):
    y=[0, 0]
    y[0] = x[0]*cos(x[1]) - 4
    y[1] = x[0]*x[1] - x[1] - 5
    return y

x = fsolve(f03, [1, 1])
print(x)

[6.50409711 0.90841421]


## 2. Optimization

#### Example 6: Optimization with constraints
<p>
<div style="width: 400px;">![example 6](example6.png)</div>

In [254]:
from scipy.optimize import minimize
funcc = lambda y: (y[0] - 1)**2 + (y[1] - 2.5)**2
cons = ({'type': 'ineq', 'fun': lambda y:  y[0] - 2 * y[1] + 2},
        {'type': 'ineq', 'fun': lambda y: -y[0] - 2 * y[1] + 6},
        {'type': 'ineq', 'fun': lambda y: -y[0] + 2 * y[1] + 2})
bnds = ((0, None), (0, None))
res = minimize(funcc, (2, 0), constraints=cons, bounds=bnds)
print(res.x)

[1.4 1.7]


#### Example 7: Optimization with constraints
<p>
<div style="width: 600px;">![example 7](example7.png)</div>

In [257]:
from scipy.optimize import minimize
fun = lambda x : (1/2)*x[0]**2+x[1]**2-x[0]*x[1]-2*x[0]-6*x[1]
cons = ({'type': 'ineq', 'fun' : lambda x: -x[0] -x[1]+2},
        {'type': 'ineq', 'fun' : lambda x: x[0]-2*x[1]+2},
        {'type': 'ineq', 'fun' : lambda x: -2*x[0]-x[1]+3})
bnds = ((None, None), (None, None))
res = minimize(fun, (0, 0), bounds=bnds, constraints=cons)
print(res.x)
# why res.x??

[0.66666667 1.33333333]


#### Example 8: Minimum-Variance Portfolio
<p>
<div style="width: 400px;">![example 8](example8.png)</div>

In [217]:
from scipy.optimize import minimize
import numpy as np
R=np.matrix([0.07,0.08,0.09,0.1])
#x0=np.matrix([0.1,0.1,0.1,0.1])
#x0=np.array([0.1,0.1,0.1,0.1])
x0=np.array([[0.1,0.1,0.1,0.1]])
R0=0.077
sigma=np.matrix([[0.0225, 0.009, 0.013125, 0.01125],
                 [0.009, 0.04, 0.019, 0.006],
                 [0.013125, 0.019, 0.0625, 0.01125],
                 [0.01125, 0.006, 0.01125, 0.09]])
def fun(w, sigma):
    Mw=np.matrix(w)    
    return (Mw*sigma*(Mw.T))[0,0]

cons = ({'type': 'eq', 'fun': lambda x: (np.matrix(x)*(R.T))[0,0]-R0},
        {'type': 'eq', 'fun': lambda x: np.sum(x)-1.0})
bnds = ((0, 1), )*4
res = minimize(fun, x0, args=sigma, bounds=bnds, constraints=cons)
res.x

array([0.57053491, 0.26145849, 0.06547829, 0.10252831])

In [247]:
from scipy.optimize import minimize
import numpy as np
R=np.array([[0.07,0.08,0.09,0.1]])
x0=[0.1,0.1,0.1,0.1]
#x0=np.array([0.1,0.1,0.1,0.1])
#x0=np.array([[0.1,0.1,0.1,0.1]])
R0=0.077
sigma=np.array([[0.0225, 0.009, 0.013125, 0.01125],
                 [0.009, 0.04, 0.019, 0.006],
                 [0.013125, 0.019, 0.0625, 0.01125],
                 [0.01125, 0.006, 0.01125, 0.09]])
def fun(w, sigma):
    return (w@sigma@(w))

cons = ({'type': 'eq', 'fun': lambda x: (x@(R.T))[0]-R0},
        {'type': 'eq', 'fun': lambda x: np.sum(x)-1.0})
bnds = ((0, 1), )*4

res = minimize(fun, x0, args=sigma, bounds=bnds, constraints=cons)
res.x

array([0.57053491, 0.26145849, 0.06547829, 0.10252831])

In [14]:
from scipy.optimize import minimize
import numpy as np
R=np.array([0.07,0.08,0.09,0.1])
x0=[0.1,0.1,0.1,0.1]
#x0=np.array([0.1,0.1,0.1,0.1])
#x0=np.array([[0.1,0.1,0.1,0.1]])
R0=0.077
sigma=np.array([[0.0225, 0.009, 0.013125, 0.01125],
                 [0.009, 0.04, 0.019, 0.006],
                 [0.013125, 0.019, 0.0625, 0.01125],
                 [0.01125, 0.006, 0.01125, 0.09]])
def fun(w, sigma):
    return (w@sigma@(w.T))

cons = ({'type': 'eq', 'fun': lambda x: (x@(R.T))-R0},
        {'type': 'eq', 'fun': lambda x: np.sum(x)-1.0})
bnds = ((0, 1), )*4

res = minimize(fun, x0, args=sigma, bounds=bnds, constraints=cons)
[a, b, c, d] = res.x
print(a, b, c, d)
print(res.x)

0.5705349078765326 0.26145849193435333 0.06547829250169435 0.10252830768741955
[0.57053491 0.26145849 0.06547829 0.10252831]


In [240]:
kkk = np.array([[0.0225, 0.009, 0.013125, 0.01125],
                 [0.009, 0.04, 0.019, 0.006],
                 [0.013125, 0.019, 0.0625, 0.01125],
                 [0.01125, 0.006, 0.01125, 0.09]])
w = np.array([0.1,0.1,0.1,0.1])
R=np.array([0.07,0.08,0.09,0.1])
w@kkk@(w.T)
(w.T).shape
print(w@R)

0.034


In [250]:
pp = np.array([1, 2, 3, 4])
ll = np.array([1, 1, 1, 1])
pp.T@ll.T

10

## 3. Differentiation

In [16]:
import pandas as pd
from scipy.stats import norm
from scipy.misc import derivative
from math import log, sqrt, exp

def BS_EuroCallV(S,K,r,q,sigma,T):
   d1=(log(S/K)+(r-q+sigma**2/2)*T)/(sigma*sqrt(T))
   d2=d1-sigma*sqrt(T)
   c=S*exp(-q*T)*norm.cdf(d1)-K*exp(-r*T)*norm.cdf(d2)   
   return c
def BS_EuroCallDelta(S,K,r,q,sigma,T):
   d1=(log(S/K)+(r-q+sigma**2/2)*T)/(sigma*sqrt(T))
   Delta=exp(-q*T)*norm.cdf(d1)
   return Delta
def BS_EuroCallVega(S,K,r,q,sigma,T):
   d1=(log(S/K)+(r-q+sigma**2/2)*T)/(sigma*sqrt(T))
   Vega=S*exp(-q*T)*norm.pdf(d1)*sqrt(T)
   return Vega

data=pd.read_csv('dataset01.csv',header=0)
data['BS']=data[['S','K','r','q','sigma','T']].apply(lambda x: BS_EuroCallV(*x), axis=1)
data['Delta'] = data[['S','K','r','q','sigma','T']].apply(lambda x: derivative(lambda s: BS_EuroCallV(s, *(x[1:7])), x[0], dx=0.1), axis=1)
'''
data['Delta calc']=data[['S','K','r','q','sigma','T']].apply(lambda x: derivative(lambda s: BS_EuroCallV(s, *(x[1:])), x[0], dx)
data['Delta']=data[['S','K','r','q','sigma','T']].apply(lambda x: BS_EuroCallDelta(*x), axis=1)
data['Delta2']=data[['S','K','r','q','sigma','T']].apply(
    lambda x: derivative(lambda s: BS_EuroCallV(s,*(x[1:])),x[0],dx=0.01), axis=1)
    '''
data['Vega']=data[['S','K','r','q','sigma','T']].apply(lambda x: BS_EuroCallVega(*x), axis=1)
data['Vega2']=data[['S','K','r','q','sigma','T']].apply(
    lambda x: derivative(lambda s: BS_EuroCallV(*(x[0:4]), s, x[5]),x[4],dx=0.01), axis=1)

data


,S,K,r,q,sigma,T,c,BS,Delta,Vega,Vega2
0,1403,1350,0.0534,0.0118,0.260,0.102778,80.828,80.827847,0.709677,153.643372,153.615931
1,1403,1375,0.0534,0.0118,0.267,0.102778,66.084,66.084173,0.627880,169.821334,169.811796
2,1403,1400,0.0534,0.0118,0.231,0.102778,45.894,45.894142,0.548545,177.856491,177.855164
3,1403,1425,0.0534,0.0118,0.213,0.102778,30.955,30.955446,0.447307,177.688245,177.682856
4,1403,1450,0.0534,0.0118,0.198,0.102778,19.224,19.224057,0.336832,164.091086,164.051102


TypeError: 'numpy.float64' object is not callable

## 4. Integration

#### `scipy.integrate.quad` for integral of

$$
\int_a^b f(x) dx
$$

https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html

#### Example: Compute $\displaystyle \int_0^4 x^2 dx$

In [21]:
from scipy import integrate
x2 = lambda x: x**2
print(integrate.quad(x2, 0, 4)[0])
print(4**3 / 3.)  # analytical result

21.333333333333336
21.333333333333332


#### Example: Compute $\displaystyle \int_0^\infty e^{-x} dx$

In [20]:
from scipy import integrate
import numpy as np
invexp = lambda x: np.exp(-x)
integrate.quad(invexp, 0, np.inf)

(1.0000000000000002, 5.842606996763696e-11)

In [34]:
from scipy import integrate
import numpy as np
invexp = lambda x: np.exp(-x)
integrate.quad(invexp, 0, np.inf)

(1.0000000000000002, 5.842606996763696e-11)

#### `scipy.integrate.dblquad` for double integral of

$$
\int_a^b \int_{g(x)}^{f(x)} f(x,y) dy dx
$$

https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.dblquad.html

#### Example: Compute $\displaystyle \int_0^2 \int_0^1 xy^2 dy dx$

In [253]:
from scipy import integrate
f = lambda x, y: x*y**2
integrate.dblquad(f, 0, 1, lambda z: 0, lambda z: 2)
#integrate.dblquad(f, 0, 2, 0, 1) #Wrong

(0.6666666666666667, 2.2108134835808843e-14)